In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../Data/www/styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
import requests
import json

# Interacting with the Web (Part I)

The Internet is a gigantic data dump. There is all the social networking data from Facebook, Twitter, and so on. There is the news from all the traditional media sources plus Quartz, Vox, and so on. Then there is the data from organizations such as the World Bank, the Bureau of Labor Statistics, the US Census, or Chicago's Data Portal.  Finally, you have all your scientific data sources: the National Cancer Institute, the ProteinBank, or the Kyoto Gene and Genomes Encyclopedia.

How can you use Python to access those sites and retrieve data for your research, your business, or your hobby?

There are two main approaches to retrieve data from websources. The preferred approach is using **Application Program Interfaces** or APIs.  If an organization has decided to share its data, and they have the forethought and resources to do it, they will develop an API that will let you interact with their data.

If the organization does not have the forethought or resources to create an API (or if they do not want to share their data), they you have to **crawl** their website and **scrape** their data.


+++

# Application Program Interfaces

** We relied heavily for these materials on https://www.dataquest.io/blog/python-api-tutorial/**

APIs simplify the process of obtaining specific information from a data source.  You do not have to worry about figuring out the **format** in which the information is stored, or **where** the information is stored.  All of those matter are handled seamlessly by the API. 

But convenience is not the only advantage of an API. APIs are also particular useful when:

* You want a small piece of a much larger set of data. Reddit comments are one example. What if you want to just pull your own comments on Reddit? It doesn’t make much sense to download the entire Reddit database, then filter just your own comments.
    
* There is repeated computation involved. Spotify has an API that can tell you the genre of a piece of music. You could theoretically create your own classifier, and use it to categorize music, but you’ll never have as much data as Spotify does.
    
* The data is changing quickly. An example of this is stock price data. It doesn’t really make sense to regenerate a dataset and download it every minute – this will take a lot of bandwidth, and be pretty slow.
    
    
    
### Making a request

In order to learn how APIs work, we will first use the APIs developed to retrieve data on the **International Space Station (ISS)**.  The relevant APIs can be found at http://open-notify.org/.  We will first consider the API for retrieving the location (latitude and longitude) of the ISS (http://open-notify.org/Open-Notify-API/ISS-Location-Now/). The API is hosted at http://api.open-notify.org/iss-now.json. 

So, how do we make requests for information with this API?

Like standard webpages, APIs are also hosted on web servers. When you type http://www.google.com in your browser’s address bar, your computer is actually asking the http://www.google.com server for a webpage, which it then returns it to your browser for display. That action is called a `request`. APIs work much the same way, except instead of your web browser asking for a webpage, your program asks for **data**. This data is usually returned in JSON format.

There are many possible types of requests. The most common, and the one we will be using throughout this unit, is the `GET` request. A `GET` request simply accesses and downloads the webpage found at the URL you specified as an input. 

We will use the package [`requests`](http://docs.python-requests.org/en/latest/user/quickstart/) package to crawl (load) webpages and scrape (download) their contents.

In [21]:
response = requests.get("http://api.open-notify.org/iss-now.json")
print( response )
print( response.status_code )


<Response [200]>
200


Methods from the `requests` package return `Response` objects. One of the most important properties of the response is its `status code`, which is printed by default but which we can also get explicitly.

Here are some of the most common status codes you might encounter:
* 200, **OK**. Standard response for successful HTTP requests. The actual response will depend on the request method used.
* 301, **Moved Permanently**. The server is redirecting you to a different endpoint. This and all future requests should be directed to the given URL. This can happen when a company switches domain names, or an endpoint name is changed.
* 303, **See Other**. The response to the request can be found under another URI using a GET method. When received in response to a POST (or PUT/DELETE), the client should presume that the server has received the data and should issue a redirect with a separate GET message. Your web browser automatically fetches the new URL but web crawlers do not usually do this unless you specify it.
* 400, **Bad Request**. The server cannot or will not process the request due to an apparent client error (e.g., malformed request syntax, invalid request message framing, or deceptive request routing).
* 401, **Unauthorized**. Similar to `403 Forbidden`, but specifically for use when authentication is required and has failed or has not yet been provided. The response must include a WWW-Authenticate header field containing a challenge applicable to the requested resource.
* 403, **Forbidden**. The request was a valid request, but the server is refusing to respond to it. `403` error semantically means "unauthorized", i.e. the user does not have the necessary permissions for the resource.
* 404, **Not Found**. The requested resource could not be found but may be available in the future. Subsequent requests by the client are permissible.
* 500, **Internal Server Error**. A generic error message, given when an `unexpected` condition was encountered and no more specific message is suitable.
* 503, **Service Unavailable**. The server is currently unavailable (because it is overloaded or down for maintenance). Generally, this is a temporary state.
* 504, **Gateway Timeout**. The server was acting as a gateway or proxy and did not receive a timely response from the upstream server.[



More codes: http://en.wikipedia.org/wiki/List_of_HTTP_status_codes

The status code of our request was **200**. It means that all went well -- we successfully connected to the web address we wanted and downloaded its contents.

But `status codes` are not the only methods available:

In [22]:
print( response.url )

http://api.open-notify.org/iss-now.json


In [23]:
print( response.text )

{
  "iss_position": {
    "latitude": -34.73637843929847, 
    "longitude": -11.577299562992197
  }, 
  "message": "success", 
  "timestamp": 1469637223
}



This is the content format specified http://open-notify.org/Open-Notify-API/ISS-Location-Now/. It is in `json` format which means that we can easily parse it using the `json` module.

In [24]:
data = json.loads(response.text)

print( data )



{'timestamp': 1469637223, 'message': 'success', 'iss_position': {'longitude': -11.577299562992197, 'latitude': -34.73637843929847}}


**YES**. 

The method `loads()` returns json formatted data as a dictionary. We can print whatever information we need from the dictionary using the appropriate `keys`.

In [40]:
print( "The ISS current position is {0:6.3f} of latitude and {1:6.3f} of longitude.".format( 
        data['iss_position']['latitude'], 
        data['iss_position']['longitude'] ) )

The ISS current position is -34.736 of latitude and -11.577 of longitude.


### Another example

The [`Open Notify`](http://open-notify.org/) API can handle different kinds of data requests. Let's now consider the case overhead pass prediction API (http://open-notify.org/Open-Notify-API/ISS-Pass-Times/).  Before we write any code, it is important to check the requirements of the API. 

**Overview**

`The API returns a list of upcoming ISS passes for a particular location formatted as JSON.`

`As input it expects a latitude/longitude pair, altitude and how many results to return. All fields are required.`

`As output you get the same inputs back (for checking) and a time stamp when the API ran in addition to a success or failure message and a list of passes. Each pass has a duration in seconds and a rise time as a unix time stamp.`

Notice the second line. We need to provide inputs. Let's see how we can do this!

Any suggestions?

.

.

.

.

.

.

.

.

.

.

.

.


Yes, we need to read the documentation for [requests](http://docs.python-requests.org/en/master/user/quickstart/#passing-parameters-in-urls). Specifically, the documentation concerning passing parameters.

This is the relevant information: `Requests allows you to provide these arguments as a dictionary, using the params keyword argument.`

And what are the parameters we need to provide? The API's documentation has the answer:

**Input**

`This API has 2 required input values and 2 optional ones.`

`Inptut	Description	Query string	Valid Range	Units	Required?`

`Latitude	The latitude of the place to predict passes	lat	-80..80	degrees	YES`

`Longitude	The longitude of the place to predict passes	lon	-180..180	degrees	YES`

`Altitude	The altitude of the place to predict passes	alt	0..10,000	meters	No`

`Number	The number of passes to return	n	1..100	–	No`

The beauty of code documentation. If you recall, the `Overview` stated that all inputs were required. The `Inputs` section let's know that only `lat` and `lon` are actually required.  Let's try and see what happens

In [42]:
#our_location = {'lat': , 'lon': , 'alt': , 'n:' }

our_location = {'lat': 41.878, 'lon': -87.6298}

response = requests.get("http://api.open-notify.org/iss-pass.json", params = our_location)

print( response.url )
print( response.content )

http://api.open-notify.org/iss-pass.json?lat=41.878&lon=-87.6298
b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1469644816, \n    "latitude": 41.878, \n    "longitude": -87.6298, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 390, \n      "risetime": 1469666158\n    }, \n    {\n      "duration": 630, \n      "risetime": 1469671781\n    }, \n    {\n      "duration": 612, \n      "risetime": 1469677584\n    }, \n    {\n      "duration": 564, \n      "risetime": 1469683442\n    }, \n    {\n      "duration": 600, \n      "risetime": 1469689262\n    }\n  ]\n}\n'


**What have we learned?** 

That indeed providing values for `alt` and `n` is not required.  The APIs just assigns those variables default values of `100` and `5`, respectively.

We can also see that, as stated in the documentation, the output returns both the values of our inputs and the data we are requesting.

**Exercise:** It is now time for you to try to use an API on your own. The last API available at [`Open Notify`](http://open-notify.org/) returns the number of astronauts in the ISS. Write the code to access that information.

.

.

.

## Using The US Census' API

The United States Census is a decennial census mandated by the United States Constitution. The United States Census Bureau (officially the Bureau of the Census) is responsible for the United States Census.

The first census after the American Revolution was taken in 1790, under Secretary of State Thomas Jefferson; there have been 22 federal censuses since that time. The current national census was held in 2010; the next census is scheduled for 2020 and will be largely conducted using the Internet. For years between the decennial censuses, the Census Bureau issues estimates made using surveys and statistical models.

The Census Bureau has begun rolling out their datasets via [APIs](http://www.census.gov/developers/). You can find a full list of APIs [here](http://www.census.gov/data/developers/data-sets.html).  In this unit, we will focus on the [decennial census](http://www.census.gov/data/developers/data-sets/decennial-census-data.html).

Because we are dealing with US data, we will start by loading some helpful data: US city names, their states, and their geographic codes.  The relevant data is stored in `json` format in `../Data/`



In [39]:
with open('../Data/us_state_names.json') as file_in:
    states_w_codes = json.load( file_in )
    
with open('../Data/us_state_city_names.json') as file_in:
    cities_by_state = json.load( file_in )

print(states_w_codes.keys())

dict_keys(['AZ', 'WY', 'NH', 'NY', 'ME', 'NC', 'ID', 'GA', 'RI', 'UT', 'NV', 'TX', 'PR', 'MD', 'SD', 'AR', 'AK', 'CA', 'MN', 'CT', 'SC', 'WI', 'NJ', 'KS', 'MA', 'MT', 'VA', 'AL', 'FL', 'VT', 'KY', 'HI', 'NM', 'OH', 'ND', 'MO', 'CO', 'DE', 'IA', 'WV', 'OR', 'PA', 'LA', 'OK', 'IL', 'IN', 'WA', 'MI', 'MS', 'NE', 'TN'])


In [40]:
print(states_w_codes['PA'])

{'fips_state': '42', 'Name': 'Pennsylvania'}


**FIPS state codes** are numeric and two-letter alphabetic codes defined in U.S. Federal Information Processing Standard Publication ("FIPS PUB") 5-2 to identify U.S. states and certain other associated areas. The codes are used in Geographic Names Information System, overseen by the U.S. Board on Geographic Names. 

In [41]:
print(cities_by_state.keys())

dict_keys(['AZ', 'WY', 'NH', 'DC', 'NY', 'ME', 'NC', 'ID', 'GA', 'RI', 'UT', 'NV', 'TX', 'PR', 'MD', 'SD', 'AR', 'AK', 'CA', 'MN', 'CT', 'SC', 'WI', 'NJ', 'KS', 'MT', 'VA', 'AL', 'FL', 'VT', 'KY', 'HI', 'NM', 'OH', 'ND', 'MO', 'NE', 'CO', 'DE', 'IA', 'WV', 'OR', 'PA', 'LA', 'OK', 'IL', 'IN', 'WA', 'MI', 'MS', 'MA', 'TN'])


In [42]:
print(states_w_codes['MT'])
print(cities_by_state['MT']['Antelope'])

{'fips_state': '30', 'Name': 'Montana'}
{'fips_state': '30', 'gnis_city': '2407745', 'fips_city': '02050', 'type': 'CDP'}


Now that we have the basic information, we can start using the API to retrieve data. The Census Bureau has a number of helpful resources.  The [decennial census page](http://www.census.gov/data/developers/data-sets/decennial-census-data.html) constains basic instructions on how to contruct queries. There is a also a [page with examples](http://api.census.gov/data/2010/sf1/examples.html), and a page with a list of all (and I *really* mean **all**) [variable codes](http://api.census.gov/data/2010/sf1/variables.html).

But before we can do anything, we need to obtain a `key` that will identify us as the person doing the queries.

In [43]:
my_census_key = '1eb4e956ec8bdfd987960641728d0fed68589575'
census_url = 'http://api.census.gov/data/2010/sf1'

# Example from Census: http://api.census.gov/data/2010/sf1?key=b48301d897146e8f8efd9bef3c6eb1fcb864cf&get=P0010001,NAME&for=state:*

response = requests.get( census_url, params = {'key': my_census_key, 'get': 'P0010001,NAME', 'for': 'state: *'})
print(response.status_code)
print('http://api.census.gov/data/2010/sf1?key=1eb4e956ec8bdfd987960641728d0fed68589575&get=P0010001,NAME&for=state:*')
print(response.url)
print(response.text)

200
http://api.census.gov/data/2010/sf1?key=1eb4e956ec8bdfd987960641728d0fed68589575&get=P0010001,NAME&for=state:*
http://api.census.gov/data/2010/sf1?key=1eb4e956ec8bdfd987960641728d0fed68589575&get=P0010001%2CNAME&for=state%3A+%2A
[["P0010001","NAME","state"],
["4779736","Alabama","01"],
["710231","Alaska","02"],
["6392017","Arizona","04"],
["2915918","Arkansas","05"],
["37253956","California","06"],
["5029196","Colorado","08"],
["3574097","Connecticut","09"],
["897934","Delaware","10"],
["601723","District of Columbia","11"],
["18801310","Florida","12"],
["9687653","Georgia","13"],
["1360301","Hawaii","15"],
["1567582","Idaho","16"],
["12830632","Illinois","17"],
["6483802","Indiana","18"],
["3046355","Iowa","19"],
["2853118","Kansas","20"],
["4339367","Kentucky","21"],
["4533372","Louisiana","22"],
["1328361","Maine","23"],
["5773552","Maryland","24"],
["6547629","Massachusetts","25"],
["9883640","Michigan","26"],
["5303925","Minnesota","27"],
["2967297","Mississippi","28"],
["5988

We can also write queries that obtain several data sets all at once. For example, we can obtain population by age and ethnicity using the codes:

* P012A018 -- Sex By Age (White Alone) MALE 15 yrs old
* P012A038 -- Sex By Age (White Alone) MALE 35 yrs old
* P012B018 -- Sex By Age (Black Or African American Alone) MALE 15 yrs old

And we can restrict the query to a single state.

In [46]:
data_codes = ''
for i in ['P012A018', 'P012A038', 'P012B018']:
    data_codes += i + ','
data_codes += 'NAME'
print(data_codes)

state_fips = 'state:' + states_w_codes['AK']['fips_state']

response = requests.get( census_url, params = {'key': my_census_key, 'get': data_codes, 'for': state_fips})
print(response.status_code)
print(response.url)
print(response.text)

PCT012A018,PCT012A038,PCT012B018,NAME
200
http://api.census.gov/data/2010/sf1?key=1eb4e956ec8bdfd987960641728d0fed68589575&get=PCT012A018%2CPCT012A038%2CPCT012B018%2CNAME&for=state%3A02
[["PCT012A018","PCT012A038","PCT012B018","NAME","state"],
["3075","3465","188","Alaska","02"]]


We can also retrieve the population for specific cities.

In [48]:
location_codes = 'place:'
for city in ['Chicago', 'Evanston', 'Wilmette']:
    location_codes += '0'+ cities_by_state['IL'][city]['fips_city'] + ','
location_codes = location_codes[:-1]

state_fips = 'state:' + states_w_codes['IL']['fips_state']

response = requests.get( census_url, params = {'key': my_census_key, 'get': data_codes, 'for': location_codes, 'in': state_fips})

print(response.status_code)
print(response.url)
print(response.text)


200
http://api.census.gov/data/2010/sf1?key=1eb4e956ec8bdfd987960641728d0fed68589575&get=PCT012A018%2CPCT012A038%2CPCT012B018%2CNAME&for=place%3A014000%2C024582%2C082075&in=state%3A17
[["PCT012A018","PCT012A038","PCT012B018","NAME","state","place"],
["5092","11423","7592","Chicago city","17","14000"],
["233","333","114","Evanston city","17","24582"],
["237","64","3","Wilmette village","17","82075"]]


### Refactor our code 

We have written code that can retrieve specific decennial census information, however, that code is not modular or generalizable. In order to write better code it is useful to refactor our code so it is modular and generalizable.


In [ ]:
def create_query_for_census_API(ages, cities, state_code, census_key, ethnicity_code = 'A' ):
    """
    Creates a query for retrieving male populations of given ethnicity for a given set of cities
    
    input:
        ages - list : ages of male population to query
        cities - list : fips codes of cities to query 
        state_code - str : fips code of state for cities
        census_key - str : user personal key for census API
        ethnicity_code - str : ethnicity census code (A, B, C, D, H)
        
    output:
        query - dict : params for API query
    """
    
    return query

reponse = requests.get( census_url, params = create_query_for_census_API() )